In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
import pickle
import json
from rich import print

In [79]:
# Condif With Feature names
with open('src/config.json', 'r') as f:
    config_dict = json.load(f)

y_column = config_dict['y_column']
numeric_features = config_dict['numeric_features']
categorical_features = config_dict['categorical_features']
all_features = numeric_features + categorical_features

# Processed Data
with open('data/processed/ml_data.pkl', 'rb') as f:
    data_dict = pickle.load(f)

X_train = data_dict['X_train']
X_test = data_dict['X_test']
y_train = data_dict['y_train']
y_test = data_dict['y_test']
feature_names = data_dict['feature_names']
X_preprocessor = data_dict['X_preprocessor']
y_preprocessor = data_dict['y_preprocessor']

In [80]:
X_train

array([[ 0.13907964,  0.13907964, -0.00963163, ...,  0.        ,
         0.        ,  0.        ],
       [-0.50730151, -0.50730151, -0.01059335, ...,  0.        ,
         0.        ,  0.        ],
       [-0.64581176, -0.64581176, -0.01022832, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.73815192, -0.73815192, -0.01059335, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.459544  ,  1.459544  , -0.00862779, ...,  0.        ,
         0.        ,  0.        ],
       [-0.57193963, -0.57193963, -0.01017216, ...,  0.        ,
         0.        ,  0.        ]], shape=(20406, 35))

In [81]:
X_test

array([[ 0.86856695,  0.86856695, -0.01008792, ...,  0.        ,
         0.        ,  0.        ],
       [-0.2764511 , -0.2764511 , -0.01059335, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.15482146,  1.15482146, -0.01059335, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.56270561, -0.56270561, -0.01020726, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02713265, -0.02713265, -0.00975799, ...,  0.        ,
         0.        ,  0.        ],
       [ 2.2444354 ,  2.2444354 , -0.00803111, ...,  0.        ,
         0.        ,  0.        ]], shape=(5102, 35))

### Entrenamiento

In [82]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet

from src.evaluate import train_eval_model
from src.training import (
    optimize_hyperparameters,
    objective_random_forest,
    objective_lightgbm,
    objective_xgboost,
    objective_elastic_net
) 

models = []
scores = []
print('Tunning Random Forest')
best_params, score = optimize_hyperparameters(objective_random_forest, X_train, y_train.ravel())

models.append(RandomForestRegressor(**best_params))
scores.append(score)

print()
print('Tunning XGBoost')
best_params, score = optimize_hyperparameters(objective_xgboost, X_train, y_train.ravel())
models.append(XGBRegressor(**best_params))
scores.append(score)

print()
print('Tunning LGBoost')
best_params, score = optimize_hyperparameters(objective_lightgbm, X_train, y_train.ravel())
models.append(LGBMRegressor(**best_params))
scores.append(score)

print()
print('Tunning Elastic Net')
best_params, score = optimize_hyperparameters(objective_elastic_net, X_train, y_train.ravel())
models.append(ElasticNet(**best_params))
scores.append(score)

train_eval_model(models, scores, X_train, y_train, X_test, y_test, y_preprocessor = y_preprocessor)


Tunning Random Forest

  0%|          | 0/4 [00:00<?, ?it/s]

Best trial: 0. Best value: -0.0647387: 100%|██████████| 4/4 [00:48<00:00, 12.16s/it]

 Best score : -0.06473872622015502


Tunning XGBoost

...

Best trial: 0. Best value: -0.0692272: 100%|██████████| 4/4 [00:09<00:00,  2.28s/it]

 Best score : -0.06922722411183864


Tunning Elastic Net

Best trial: 2. Best value: -0.220771: 100%|██████████| 4/4 [00:11<00:00,  2.80s/it]


 Best score : -0.22077092795249992
==================== Best model ====================
ElasticNet(alpha=0.00012096012562789731, l1_ratio=0.3237744066896452,
           max_iter=2817, tol=2.550417252541119e-05)


REGRESSION METRICS
MAE: 311830655.0843
R²: 0.0052
MAPE: 0.3449
Explained Variance: 0.0055
